Water Purity Prediction

In [1]:
#importing relevant libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import rcParams

import seaborn as sns




In [ ]:
#Importing dataset
data = pd.read_csv('water_potability.csv')

#Data Dictionary 

df = pd.DataFrame(data=data.data, columns=data.feature_names)
# create a series for the label
df["target"] = data.target
#produce a summary of the dataset.
df.head()
df.describe()
df.shape
df.info()
